In [1]:
from distributed import Client
import dask.dataframe as dd
import pandas as pd

# Start Dask Cluster

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64837,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:64850,Total threads: 1
Dashboard: http://127.0.0.1:64853/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:64840,


# Original Executions


In [4]:
executions_ddf = dd.read_parquet('data/executions.parquet')

# will need to have a datetime column and date column
executions_ddf['TradeTime'] = executions_ddf['TradeTime'].astype('datetime64[ns]')
executions_ddf['TradeDate'] = executions_ddf['TradeTime'].dt.date
executions_ddf.head()


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate
0,DE0006305006,EUR,XETA,2022-09-02 07:00:09.160,3.606,0,OPENING_AUCTION,-150,2022-09-02
1,DE0006305006,EUR,XETA,2022-09-02 07:02:32.790,3.624,1,OPENING_AUCTION,-198,2022-09-02
2,DE0006305006,EUR,XETA,2022-09-02 07:03:01.573,3.622,2,OPENING_AUCTION,-85,2022-09-02
3,DE0006305006,EUR,XETA,2022-09-02 07:03:01.829,3.622,3,OPENING_AUCTION,-89,2022-09-02
4,DE0006305006,EUR,XETA,2022-09-02 07:03:48.935,3.626,4,OPENING_AUCTION,-119,2022-09-02


# Enriched Executions

In [5]:
enriched_executions_ddf = dd.read_parquet('data/enriched_executions.parquet')

# will need to have a datetime column and date column
enriched_executions_ddf['TradeTime'] = executions_ddf['TradeTime'].astype('datetime64[ns]')
enriched_executions_ddf.head()

,index,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,...,best_bid,best_ask,mid_price,slippage,best_bid_min_1s,best_ask_min_1s,mid_price_min_1s,best_bid_1s,best_ask_1s,mid_price_1s
0,164,DE0007664039,EUR,XETA,2022-09-02 07:00:09.160,142.0,264,CONTINUOUS_TRADING,15,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,165,DE0007664039,EUR,XETA,2022-09-02 07:02:32.790,142.0,265,CONTINUOUS_TRADING,65,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,166,DE0007664039,EUR,XETA,2022-09-02 07:03:01.573,142.0,266,CONTINUOUS_TRADING,11,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167,DE0007664039,EUR,XETA,2022-09-02 07:03:01.829,142.0,267,CONTINUOUS_TRADING,33,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,168,DE0007664039,EUR,XETA,2022-09-02 07:03:48.935,142.0,268,CONTINUOUS_TRADING,24,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Market Data

In [19]:
market_ddf = dd.read_parquet('data/marketdata.parquet')
market_ddf['event_timestamp'] = market_ddf['event_timestamp'].astype('datetime64[ns]')
market_ddf.head()

,event_timestamp,best_bid_size,best_bid_price,best_ask_price,best_ask_size,market_state,primary_mic,listing_id
0,2022-09-02 01:00:57.205489803,6740,23.00,24.75,91,CLOSED,XBRU,378534881
1,2022-09-02 01:00:57.210457483,26,31.80,32.50,65,CLOSED,XBRU,328259
2,2022-09-02 01:01:03.797937950,52,47.88,48.00,31,CLOSED,XBRU,324162
3,2022-09-02 01:01:03.802911163,281,30.65,33.33,300,CLOSED,XBRU,324168
4,2022-09-02 01:01:03.803207763,52,46.68,47.50,85,CLOSED,XBRU,324078


## Check for Side enrichment

In [8]:
enriched_executions_ddf[enriched_executions_ddf['Side'] == 2 ].head()

,index,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,...,best_bid,best_ask,mid_price,slippage,best_bid_min_1s,best_ask_min_1s,mid_price_min_1s,best_bid_1s,best_ask_1s,mid_price_1s
16,183,DE0007100000,EUR,XETA,2022-09-02 07:32:21.335,55.84,319,CONTINUOUS_TRADING,-33,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,917,CH0012005267,CHF,XSWX,2022-09-02 07:34:45.684,78.48,288,CONTINUOUS_TRADING,-26,2022-09-02,...,78.47,78.48,78.475,1.0,78.46,78.48,78.47,78.51,78.53,78.52
18,918,CH0012005267,CHF,XSWX,2022-09-02 07:35:48.256,78.58,289,CONTINUOUS_TRADING,-45,2022-09-02,...,78.58,78.60,78.590,0.0,78.48,78.50,78.49,78.59,78.61,78.60
19,184,DE0007100000,EUR,XETA,2022-09-02 07:36:39.810,55.81,320,CONTINUOUS_TRADING,-57,2022-09-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,920,CH0012005267,CHF,XSWX,2022-09-02 07:39:43.600,78.57,291,CONTINUOUS_TRADING,-280,2022-09-02,...,78.57,78.58,78.575,0.0,78.58,78.60,78.59,78.57,78.59,78.58


# Check for Price Enrichment Failures

In [18]:
enriched_executions_ddf[enriched_executions_ddf['best_bid'].isnull()]['id'].unique().head()

0    326330
1    326288
2    362008
3    326525
4    327489
Name: id, dtype: int64

In [35]:
market_ddf[market_ddf['listing_id'] == 327489].head()

,event_timestamp,best_bid_size,best_bid_price,best_ask_price,best_ask_size,market_state,primary_mic,listing_id


In [21]:
enriched_executions_ddf[enriched_executions_ddf['best_bid'].notnull()]['id'].unique().head()

0    323448
1    323508
2    323496
3    323472
4    324162
Name: id, dtype: int64

In [37]:
market_ddf[(market_ddf['listing_id'] == 323448) & (market_ddf['market_state'] == 'CONTINUOUS_TRADING')].head()

,event_timestamp,best_bid_size,best_bid_price,best_ask_price,best_ask_size,market_state,primary_mic,listing_id
255,2022-09-02 07:00:02.072892,217,78.50,78.55,78,CONTINUOUS_TRADING,XSWX,323448
256,2022-09-02 07:00:02.072924,217,78.50,78.55,268,CONTINUOUS_TRADING,XSWX,323448
257,2022-09-02 07:00:02.072982,600,78.46,78.55,268,CONTINUOUS_TRADING,XSWX,323448
258,2022-09-02 07:00:02.073029,600,78.45,78.55,268,CONTINUOUS_TRADING,XSWX,323448
259,2022-09-02 07:00:02.073032,1597,78.43,78.55,268,CONTINUOUS_TRADING,XSWX,323448
